In [5]:
#The goal of this project is to download all of the 2018 SAT information and append all of that information
#into a preexisting data-frame

import os
import pandas as pd
import numpy as np

os.listdir()

#This is the starter file which has some of the 2017 SAT information 
merged = pd.read_csv("./combined_2017.csv")

In [6]:
import requests

#All of the SAT urls follow the format /pdf/2018-*state*-sat-suite-assessments-annual-report.pdf
#So lets download the buggers DC will probably be a pain, so we can fix that too

states = [x.replace(" ", "-") if x != 'district of columbia' else "district-columbia" for x in list(merged["State"].str.lower())]

for state in states:
    url = "https://reports.collegeboard.org/pdf/2018-{}-sat-suite-assessments-annual-report.pdf".format(state)
    r = requests.get(url, allow_redirects=True)
    open('2018_SAT_{}_Data.pdf'.format(state), 'wb').write(r.content)

In [7]:
#Just a little check to see if all of the files downloaded correctly.

count = 0
for file in os.listdir():
    if file[-3:] == "pdf":
        print(count,file)
        count += 1

0 2018_SAT_massachusetts_Data.pdf
1 2018_SAT_missouri_Data.pdf
2 2018_SAT_nebraska_Data.pdf
3 2018_SAT_kentucky_Data.pdf
4 2018_SAT_new-hampshire_Data.pdf
5 2018_SAT_new-york_Data.pdf
6 2018_SAT_oklahoma_Data.pdf
7 2018_SAT_nevada_Data.pdf
8 2018_SAT_washington_Data.pdf
9 2018_SAT_new-jersey_Data.pdf
10 2018_SAT_hawaii_Data.pdf
11 2018_SAT_west-virginia_Data.pdf
12 2018_SAT_arizona_Data.pdf
13 2018_SAT_oregon_Data.pdf
14 2018_SAT_new-mexico_Data.pdf
15 2018_SAT_maryland_Data.pdf
16 2018_SAT_wyoming_Data.pdf
17 2018_SAT_georgia_Data.pdf
18 2018_SAT_ohio_Data.pdf
19 2018_SAT_maine_Data.pdf
20 2018_SAT_vermont_Data.pdf
21 2018_SAT_illinois_Data.pdf
22 2018_SAT_north-carolina_Data.pdf
23 2018_SAT_alaska_Data.pdf
24 2018_SAT_utah_Data.pdf
25 2018_SAT_pennsylvania_Data.pdf
26 2018_SAT_idaho_Data.pdf
27 2018_SAT_texas_Data.pdf
28 2018_SAT_mississippi_Data.pdf
29 2018_SAT_michigan_Data.pdf
30 2018_SAT_alabama_Data.pdf
31 2018_SAT_iowa_Data.pdf
32 2018_SAT_connecticut_Data.pdf
33 2018_SAT_india

In [8]:
import slate3k as slate

def pdf_view(file):
    mine  = []
    with open(file, 'rb') as f:
        doc = slate.PDF(f)
    
    pdf_miner =  [x for x in doc[3].split("\n") if x != ""]  
    for i in range(len(pdf_miner)):
        mine.append(pdf_miner[i].lstrip())
    
    return(mine)

pdf_view("2018_SAT_massachusetts_Data.pdf")

['SAT Participation and Performance',
 'Number',
 'Percent',
 'Total',
 'ERW',
 'Math',
 'Both',
 'ERW',
 'Math',
 'None',
 'Female',
 'Male',
 'No Response',
 '18%',
 '19%',
 '67%',
 '58%',
 '79%',
 '80%',
 '64%',
 '56%',
 '576',
 '550',
 '563',
 '561',
 '1140',
 '1112',
 '0%',
 '47%',
 '53%',
 '6',
 '28,125',
 '31,251',
 'Number',
 'Percent',
 'Total',
 'ERW',
 'Math',
 'Both',
 'ERW',
 'Math',
 'None',
 'American Indian/Alaska Native',
 'Asian',
 'Black/African American',
 'Hispanic/Latino',
 'Native Hawaiian/Other Pacific Islander',
 'White',
 'Two or More Races',
 'No Response',
 '35%',
 '16%',
 '11%',
 '27%',
 '41%',
 '45%',
 '10%',
 '25%',
 '43%',
 '63%',
 '69%',
 '42%',
 '36%',
 '32%',
 '82%',
 '45%',
 '62%',
 '82%',
 '88%',
 '73%',
 '57%',
 '52%',
 '84%',
 '72%',
 '40%',
 '62%',
 '68%',
 '42%',
 '34%',
 '29%',
 '76%',
 '42%',
 '512',
 '572',
 '577',
 '520',
 '497',
 '485',
 '646',
 '519',
 '517',
 '575',
 '582',
 '522',
 '500',
 '488',
 '594',
 '529',
 '1029',
 '1147',
 '1159'

In [9]:
#There are other packages to help with pdf data scraping, but this one is easy for text. However, sorry about 
#warnings, Pdfminer3k sets directly to the Python root logger :(

'''
Here is the stuff I need to mine

"State" 
"SAT_Participation"
"SAT_English"
"SAT_Math"
"SAT_Total"

'''

import slate3k as slate

def pdf_mine(file):
    mine  = []
    with open(file, 'rb') as f:
        doc = slate.PDF(f)
    
    pdf_miner =  [x for x in doc[2].split("\n") if x != ""] 
    #Sice these 51 pdf are exactly the same, but for the state SAT data, we can set the location
    #and mine from the lists
    for i in range(len(pdf_miner)):
        if i == 1:
            mine.append(pdf_miner[i].lstrip())
        elif pdf_miner[i][-14:] == '% of graduates': 
            mine.append(pdf_miner[i].lstrip())

    pdf_miner = [x for x in doc[3].split("\n") if x != ""]
    for i in range(len(pdf_miner)):
        if pdf_miner[i].lstrip() == "Took Essay¹": #That little 1 helps!
            mine.append(pdf_miner[i + 12].lstrip())
            mine.append(pdf_miner[i + 10].lstrip())
            mine.append(pdf_miner[i + 14].lstrip())
    
    print(mine)
    return(mine)

print(pdf_mine('2018_SAT_massachusetts_Data.pdf'))

['Massachusetts', '80% of graduates', '562', '563', '1125']
['Massachusetts', '80% of graduates', '562', '563', '1125']


In [10]:
#The idea of this workflow is to set an empty dictionary and append all of the different SAT information 
#To that file.

SAT_2018 = {}
for file in os.listdir():
    if file[0] != "A" and file[-3:] == "pdf":
        try:
            mine = pdf_mine(file)
            if file[-3:] == "pdf":
                SAT_2018[mine[0]] = {
                    "State":mine[0], 

                    #I left the '% of graduates' below for inspection, need
                    #to strip now

                    "SAT_2018_Participation":float(mine[1].split("%")[0])/100,
                    "SAT_2018_English":int(mine[2]),
                    "SAT_2018_Math":int(mine[3]),
                    "SAT_2018_Total":int(mine[4])}
        except:
            print("something went wrong with {}".format(file))

['Massachusetts', '80% of graduates', '562', '563', '1125']


['Missouri', '4% of graduates', '633', '629', '1262']


['Nebraska', '3% of graduates', '629', '623', '1252']


['Kentucky', '4% of graduates', '630', '618', '1248']


['New Hampshire', '96% of graduates', '535', '528', '1063']


...

['Kansas', '4% of graduates', '633', '631', '1265']


['Delaware', '100% of graduates', '505', '492', '998']


['Florida', '97% of graduates', '522', '493', '1014']


['Minnesota', '4% of graduates', '643', '655', '1298']


['South Carolina', '55% of graduates', '547', '523', '1070']


In [13]:
sat2018 = pd.DataFrame.from_dict(SAT_2018, orient='index').reset_index()
sat2018.drop("index", axis = 1, inplace = True)
sat2018.head()

,State,SAT_2018_Participation,SAT_2018_English,SAT_2018_Math,SAT_2018_Total
0,Alabama,0.06,595,571,1166
1,Alaska,0.43,562,544,1106
2,Arizona,0.29,577,572,1149
3,Arkansas,0.05,592,576,1169
4,California,0.60,540,536,1076


In [19]:
merged_2017_18 = pd.merge(merged, sat2018, on='State')
merged_2017_18.drop("Unnamed: 0", axis = 1, inplace = True)


In [21]:
#Done!
merged_2017_18.head()

,State,SAT_2017_Participation,SAT_2017_English,SAT_2017_Math,SAT_2017_Total,ACT_2017_Participation,ACT_2017_English,ACT_2017_Math,ACT_2017_Reading,ACT_2017_Science,ACT_2017_Composite,SAT_2018_Participation,SAT_2018_English,SAT_2018_Math,SAT_2018_Total
0,Alabama,0.05,593,572,1165,1.00,18.9,18.4,19.7,19.4,19.2,0.06,595,571,1166
1,Alaska,0.38,547,533,1080,0.65,18.7,19.8,20.4,19.9,19.8,0.43,562,544,1106
2,Arizona,0.30,563,553,1116,0.62,18.6,19.8,20.1,19.8,19.7,0.29,577,572,1149
3,Arkansas,0.03,614,594,1208,1.00,18.9,19.0,19.7,19.5,19.4,0.05,592,576,1169
4,California,0.53,531,524,1055,0.31,22.5,22.7,23.1,22.2,22.8,0.60,540,536,1076
